In [12]:
import pandas as pd
from tabula import read_pdf

In [16]:
# this process extracts the multiple columns of values on each page
# because of how the PDF is designed, this results in the headers
# being truncated; we gather those separately
data_values = read_pdf(
    'source_files/2022-assistance-listing-catalog.pdf',
    output_format='dataframe',
    pandas_options={'header': None},
    pages=['151-208'],
    stream=True,
    multiple_tables=False,
    area=[[55, 60, 735, 90], [55,310,735,340]],
    encoding="utf-8"
)
data_values = data_values[0]

# remove a handful of outliers from page one that precede the template
# these are only valid for the 2022 PDF
data_values = data_values.drop(range(0,19))
data_values = data_values.drop(55).reset_index(drop=True)

# load in the 2022 functions list
functions_df = pd.read_csv('source_files/2022-functions-list.csv', header=None)

In [17]:
parent = ''
child = ''

parent_col = []
child_col = []
remove_rows = []
function_index = 0
i = -1
for row in data_values[0]:
    i += 1
    if i in [2110,2255]: # ignore on two-line sub-function 
        parent_col.append('')
        child_col.append('')
        remove_rows.append(i)
        continue
    if row[0].isdigit():
        parent_col.append(parent)
        child_col.append(child)
        continue
    if row.isupper():
        parent_col.append('')
        child_col.append('')
        remove_rows.append(i)
        continue
    r = functions_df.loc[function_index]
    function_index+=1
    parent = r[0]
    child = r[1]
    parent_col.append('')
    child_col.append('')
    remove_rows.append(i)

data_values[1] = parent_col
data_values[2] = child_col
data_values = data_values.drop(remove_rows).reset_index(drop=True)

In [20]:
data_values.to_csv('data_extracts/db_program_to_function_sub_function.csv', index=False, header=False)